In [ ]:
# 查看当前挂载的数据集目录
!ls /home/kesci/input/

In [ ]:
# 查看个人持久化工作区文件
!ls /home/kesci/work/

In [ ]:
# 查看当前kernerl下的package
!pip list --format=columns

In [ ]:
# 显示cell运行时长
%load_ext klab-autotime

In [ ]:
# 以下代码是保存最后1e的数据到工作区，因为训练集数据量过大，直接通过跳行的方式读取会很慢

In [ ]:
import pandas as pd
import gc

chunksize = 300000000

data_reader = pd.read_csv('', header=None, usecols=[0, 1, 3, 4], names=['query_id', 'query', 'title', 'label'],
                          chunksize=chunksize)

post_10kw_data = None

for index, data in enumerate(data_reader):
    if index == 3:
        post_10kw_data = data
    else:
        del data
        gc.collect()

# 单独保存后1e的数据 用于训练词向量 训练模型等
post_10kw_data.to_csv('/home/kesci/work/word2vec/post_10kw.csv', index=None)

In [ ]:
# final word2vec 训练

In [ ]:
import time

import pandas as pd
import gc

from gensim.models.callbacks import CallbackAny2Vec
from gensim.models.word2vec import Word2Vec


class EpochSaver(CallbackAny2Vec):
    '''用于保存模型, 打印损失函数等等'''

    def __init__(self):
        self.epoch = 0
        self.pre_loss = 0
        self.best_loss = 999999999.9
        self.since = time.time()

    def on_epoch_end(self, model):
        self.epoch += 1
        cum_loss = model.get_latest_training_loss()  # 返回的是从第一个epoch累计的
        epoch_loss = cum_loss - self.pre_loss
        time_taken = time.time() - self.since
        print("Epoch %d, loss: %.2f, time: %dmin %ds" %
              (self.epoch, epoch_loss, time_taken // 60, time_taken % 60))
        if self.best_loss > epoch_loss:
            self.best_loss = epoch_loss
            print("Better model. Best loss: %.2f" % self.best_loss)
            model.save('./word2vec/word2vec_100.model')
            model.wv.save_word2vec_format('./word2vec/word2vec_100.bin', binary=True)
            print("Model %s save done!" % './word2vec/word2vec_100.model')

        self.pre_loss = cum_loss
        self.since = time.time()


def log(log: str):
    print(log)


def time_log(time_elapsed):
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))  # 打印时间


def log_event(event: str):
    log(event)


def get_sentence(train_path_1: str, train_path2: str, test_2kw_path: str, test_1e_path: str, chunk_size: int):
    for index, data in enumerate(
            pd.read_csv(train_path_1, chunksize=chunk_size, header=None, usecols=[1, 3], names=['query', 'title'],
                        nrows=200000000)):

        print(f'train 1 path = {train_path_1} index = {index}')
        query_df = data['query'].drop_duplicates()

        query_list = query_df.values.tolist()
        for item in query_list:
            yield item

        title_list = data['title'].values.tolist()
        for item in title_list:
            yield item
        del query_list, title_list, query_df, data
        gc.collect()

    for index, data in enumerate(
            pd.read_csv(train_path2, chunksize=chunk_size)):

        print(f'train 2 path = {train_path2} index = {index}')
        query_df = data['query'].drop_duplicates()

        query_list = query_df.values.tolist()
        for item in query_list:
            yield item

        title_list = data['title'].values.tolist()
        for item in title_list:
            yield item

        del query_list, title_list, query_df, data
        gc.collect()

    for index, data in enumerate(
            pd.read_csv(test_2kw_path, chunksize=chunk_size, header=None, usecols=[1, 3], names=['query', 'title'])):
        print(f'test path = {test_2kw_path} index = {index}')

        query_df = data['query'].drop_duplicates()

        query_list = query_df.values.tolist()
        for item in query_list:
            yield item

        title_list = data['title'].values.tolist()
        for item in title_list:
            yield item

        del query_list, title_list, query_df, data
        gc.collect()

    for index, data in enumerate(
            pd.read_csv(test_1e_path, chunksize=chunk_size, header=None, usecols=[1, 3], names=['query', 'title'])):
        print(f'path = {test_1e_path} index = {index}')
        query_df = data['query'].drop_duplicates()

        query_list = query_df.values.tolist()
        for item in query_list:
            yield item

        title_list = data['title'].values.tolist()
        for item in title_list:
            yield item

        del query_list, title_list, query_df, data
        gc.collect()


class Sentence(object):
    def __init__(self, train_path_1: str, train_path2: str, test_2kw_path: str, test_1e_path: str, chunk_size: int):
        self.train_path_1 = train_path_1
        self.train_path2 = train_path2
        self.test_2kw_path = test_2kw_path
        self.test_1e_path = test_1e_path
        self.chunk_size = chunk_size

    def __iter__(self):
        for sentence in get_sentence(self.train_path_1, self.train_path2, self.test_2kw_path, self.test_1e_path,
                                     self.chunk_size):
            seg_list = sentence.split()
            yield seg_list


sentences = Sentence('/home/kesci/input/bytedance/train_final.csv', '/home/kesci/work/word2vec/post_10kw.csv',
                     '/home/kesci/input/bytedance/test_final_part1.csv',
                     '/home/kesci/input/bytedance/bytedance_contest.final_2.csv',
                     chunk_size=5000000)

word2vec_dim = 100

model = Word2Vec(size=word2vec_dim, window=5, sg=1, min_count=1, workers=4)

model.build_vocab(sentences)
model.train(sentences, total_examples=model.corpus_count, epochs=10, compute_loss=True, report_delay=5 * 60,
            callbacks=[EpochSaver()])

In [ ]:
# 训练fasttext gensim中fasttext不能打印loss 就没有callback

In [ ]:
import pandas as pd
from gensim.models.fasttext import FastText
import gc


def log(log: str):
    print(log)


def time_log(time_elapsed):
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))  # 打印时间


def log_event(event: str):
    log(event)


def get_sentence(train_path_1: str, train_path2: str, test_2kw_path: str, test_1e_path: str, chunk_size: int):
    for index, data in enumerate(
            pd.read_csv(train_path_1, chunksize=chunk_size, header=None, usecols=[1, 3], names=['query', 'title'],
                        nrows=200000000)):

        print(f'train 1 path = {train_path_1} index = {index}')
        query_df = data['query'].drop_duplicates()

        query_list = query_df.values.tolist()
        for item in query_list:
            yield item

        title_list = data['title'].values.tolist()
        for item in title_list:
            yield item
        del query_list, title_list, query_df, data
        gc.collect()

    for index, data in enumerate(
            pd.read_csv(train_path2, chunksize=chunk_size)):

        print(f'train 2 path = {train_path2} index = {index}')
        query_df = data['query'].drop_duplicates()

        query_list = query_df.values.tolist()
        for item in query_list:
            yield item

        title_list = data['title'].values.tolist()
        for item in title_list:
            yield item

        del query_list, title_list, query_df, data
        gc.collect()

    for index, data in enumerate(
            pd.read_csv(test_2kw_path, chunksize=chunk_size, header=None, usecols=[1, 3], names=['query', 'title'])):
        print(f'test path = {test_2kw_path} index = {index}')

        query_df = data['query'].drop_duplicates()

        query_list = query_df.values.tolist()
        for item in query_list:
            yield item

        title_list = data['title'].values.tolist()
        for item in title_list:
            yield item

        del query_list, title_list, query_df, data
        gc.collect()

    for index, data in enumerate(
            pd.read_csv(test_1e_path, chunksize=chunk_size, header=None, usecols=[1, 3], names=['query', 'title'])):
        print(f'path = {test_1e_path} index = {index}')
        query_df = data['query'].drop_duplicates()

        query_list = query_df.values.tolist()
        for item in query_list:
            yield item

        title_list = data['title'].values.tolist()
        for item in title_list:
            yield item

        del query_list, title_list, query_df, data
        gc.collect()


class Sentence(object):
    def __init__(self, train_path_1: str, train_path2: str, test_2kw_path: str, test_1e_path: str, chunk_size: int):
        self.train_path_1 = train_path_1
        self.train_path2 = train_path2
        self.test_2kw_path = test_2kw_path
        self.test_1e_path = test_1e_path
        self.chunk_size = chunk_size

    def __iter__(self):
        for sentence in get_sentence(self.train_path_1, self.train_path2, self.test_2kw_path, self.test_1e_path,
                                     self.chunk_size):
            seg_list = sentence.split()
            yield seg_list


sentences = Sentence('/home/kesci/input/bytedance/train_final.csv', '/home/kesci/work/word2vec/post_10kw.csv',
                     '/home/kesci/input/bytedance/test_final_part1.csv',
                     '/home/kesci/input/bytedance/bytedance_contest.final_2.csv',
                     chunk_size=5000000)

word2vec_dim = 100

model = FastText(size=word2vec_dim, window=5, sg=1, min_count=1, workers=4)

model.build_vocab(sentences)
model.train(sentences, total_examples=model.corpus_count, epochs=5)

model.save('./fasttext/fasttext_100.model')
model.wv.save_word2vec_format('./fasttext/fasttext_100.bin', binary=True)

In [ ]:
# 以下cell是制作词表与保存词表 前2e数据和后1e数据和test1 test2

In [ ]:
import pandas as pd
import gc
from tqdm import tqdm


words_dict = {}


text_data1 = pd.read_csv('/home/kesci/input/bytedance/train_final.csv', usecols=[1, 3],
                         names=['query', 'title'], nrows=200000000)


query_list = text_data1['query'].drop_duplicates().values.tolist()

for item in tqdm(query_list):
    query = item.split()
    for word in query:
        words_dict[word] = words_dict.get(word, 0) + 1

del query_list
gc.collect()

title_list = text_data1['title'].drop_duplicates().values.tolist()

for item in tqdm(title_list):
    title = item.split()
    for word in title:
        words_dict[word] = words_dict.get(word, 0) + 1

del title_list

del text_data1
gc.collect()

text_data2 = pd.read_csv('/home/kesci/work/word2vec/post_10kw.csv')


query_list = text_data2['query'].drop_duplicates().values.tolist()

for item in tqdm(query_list):
    query = item.split()
    for word in query:
        words_dict[word] = words_dict.get(word, 0) + 1

del query_list
gc.collect()

title_list = text_data2['title'].drop_duplicates().values.tolist()

for item in tqdm(title_list):
    title = item.split()
    for word in title:
        words_dict[word] = words_dict.get(word, 0) + 1

del title_list
del text_data2
gc.collect()

test = pd.read_csv('/home/kesci/input/bytedance/test_final_part1.csv', usecols=[1, 3],
                   names=['query', 'title'])


query_list = test['query'].drop_duplicates().values.tolist()

for item in tqdm(query_list):
    query = item.split()
    for word in query:
        words_dict[word] = words_dict.get(word, 0) + 1

del query_list
gc.collect()

title_list = test['title'].drop_duplicates().values.tolist()

for item in tqdm(title_list):
    title = item.split()
    for word in title:
        words_dict[word] = words_dict.get(word, 0) + 1

del title_list
del test
gc.collect()

test2 = pd.read_csv('/home/kesci/input/bytedance/bytedance_contest.final_2.csv', usecols=[1, 3],
                    names=['query', 'title'])


query_list = test2['query'].drop_duplicates().values.tolist()

for item in tqdm(query_list):
    query = item.split()
    for word in query:
        words_dict[word] = words_dict.get(word, 0) + 1

del query_list
gc.collect()

title_list = test2['title'].drop_duplicates().values.tolist()

for item in tqdm(title_list):
    title = item.split()
    for word in title:
        words_dict[word] = words_dict.get(word, 0) + 1

del title_list
del test2
gc.collect()

In [ ]:
min_count = 1

print(f'words_dict len = {len(words_dict)}')
words = {i: j for i, j in list(words_dict.items()) if j >= min_count}
id2words = {i + 2: j for i, j in enumerate(words)}  # padding: 0, unk: 1
words2id = {j: i for i, j in list(id2words.items())}
print(f'words2id len = {len(words2id)}')

# 保存word2id
import pickle

# 保存词表
with open('/home/kesci/work/sunrui/NN_second_2e/second_6kw_nn_sim.pkl','wb') as f:
    pickle.dump(words2id,f)

In [ ]:
# 以下的cell使用训练好的词向量和使用的词表 制作神经网络需要的词向量矩阵 使用100维的word2vec和fasttext拼接

In [ ]:
from gensim.models import KeyedVectors

word2vec_file = '/home/kesci/work/word2vec/word2vec_100.bin' # word2vec
fast_text_file = './fasttext/fasttext_100.bin'  # fasttext
EMBEDDING_DIM = 200

emb_list = []
w2v_model = KeyedVectors.load_word2vec_format(word2vec_file, binary=True)
fasttext_model = KeyedVectors.load_word2vec_format(fast_text_file, binary=True)

import numpy as np

num_words = len(words2id) + 2

embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
from tqdm import tqdm
for word, i in tqdm(words2id.items()):  # 因为训练的词向量没用过滤低频词 所以都可以命中
    w2v = w2v_model[word]
    fasttext = fasttext_model[word]
    embedding_matrix[i] = np.concatenate([w2v, fasttext])

np.save('/home/kesci/work/sunrui/NN_second_2e/word2vec_fasttext_6kw_nn_sim.npy', embedding_matrix)

In [ ]:
# 以下的cell是使用第2e的数据训练纯文本的esim网络 在本次比赛中选择第四轮的模型
# 这个模型即用于计算相似度特征，又用于计算用于最后的模型融合

In [ ]:
import math
import os
import pickle
from collections import OrderedDict

import numpy as np
import pandas as pd
import tensorflow as tf
from keras import Input, Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Embedding, Dense, Dropout, Lambda, concatenate, GlobalAveragePooling1D, subtract, multiply, \
    TimeDistributed, LSTM, Bidirectional
from keras.regularizers import l2
from tensorflow import set_random_seed

from keras.backend.tensorflow_backend import set_session

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
set_session(sess)

seed = 2019

set_random_seed(seed)  # Tensorflow
np.random.seed(seed)  # NumPy

W2V_DIM = 200

max_seq_len = 25
epochs = 20

from keras.callbacks import Callback
import keras
import keras.backend as K
from keras.engine import Layer

# esim使用的attention层 支持mask
class DotProductAttention(Layer):
    def __init__(self, return_attend_weight=False, keep_mask=True, **kwargs):
        self.return_attend_weight = return_attend_weight
        self.keep_mask = keep_mask
        self.supports_masking = True
        super(DotProductAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        input_shape_a, input_shape_b = input_shape

        if len(input_shape_a) != 3 or len(input_shape_b) != 3:
            raise ValueError('Inputs into DotProductAttention should be 3D tensors')

        if input_shape_a[-1] != input_shape_b[-1]:
            raise ValueError('Inputs into DotProductAttention should have the same dimensionality at the last axis')

    def call(self, inputs, mask=None):
        assert isinstance(inputs, list)
        inputs_a, inputs_b = inputs

        if mask is not None:
            mask_a, mask_b = mask
        else:
            mask_a, mask_b = None, None

        e = K.exp(K.batch_dot(inputs_a, inputs_b, axes=2))  # similarity between a & b

        # apply mask before normalization (softmax)
        if mask_a is not None:
            e *= K.expand_dims(K.cast(mask_a, K.floatx()), 2)
        if mask_b is not None:
            e *= K.expand_dims(K.cast(mask_b, K.floatx()), 1)

        e_b = e / K.cast(K.sum(e, axis=2, keepdims=True) + K.epsilon(), K.floatx())  # attention weight over b
        e_a = e / K.cast(K.sum(e, axis=1, keepdims=True) + K.epsilon(), K.floatx())  # attention weight over a

        if self.return_attend_weight:
            return [e_b, e_a]

        a_attend = K.batch_dot(e_b, inputs_b, axes=(2, 1))  # a attend to b
        b_attend = K.batch_dot(e_a, inputs_a, axes=(1, 1))  # b attend to a
        return [a_attend, b_attend]

    def compute_mask(self, inputs, mask=None):
        if self.keep_mask:
            return mask
        else:
            return [None, None]

    def compute_output_shape(self, input_shape):
        if self.return_attend_weight:
            input_shape_a, input_shape_b = input_shape
            return [(input_shape_a[0], input_shape_a[1], input_shape_b[1]),
                    (input_shape_a[0], input_shape_a[1], input_shape_b[1])]
        return input_shape

# SWA算法
class SWA(Callback):
    def __init__(self, checkpoint_dir, model_name, swa_start=1):
        super(SWA, self).__init__()
        self.checkpoint_dir = checkpoint_dir
        self.model_name = model_name
        self.swa_start = swa_start
        self.swa_model = None  # the model that we will use to store the average of the weights once SWA begins

    def on_train_begin(self, logs=None):
        self.epoch = 0
        self.swa_n = 0
        # self.swa_model = copy.deepcopy(self.model)  # make a copy of the model we're training
        # Note: I found deep copy of a model with customized layer would give errors
        self.swa_model = keras.models.clone_model(self.model)
        self.swa_model.set_weights(self.model.get_weights())  # see: https://github.com/keras-team/keras/issues/1765

    def on_epoch_end(self, epoch, logs=None):
        if (self.epoch + 1) >= self.swa_start:
            self.update_average_model()
            self.swa_n += 1

        self.epoch += 1

    def update_average_model(self):
        alpha = 1. / (self.swa_n + 1)
        for layer, swa_layer in zip(self.model.layers, self.swa_model.layers):
            weights = []
            for w1, w2 in zip(swa_layer.get_weights(), layer.get_weights()):
                weights.append((1 - alpha) * w1 + alpha * w2)
            swa_layer.set_weights(weights)

    def on_train_end(self, logs=None):
        print('Logging Info - Saving SWA model checkpoint: %s_swa.hdf5\n' % self.model_name)
        self.swa_model.save_weights(os.path.join(self.checkpoint_dir, '{}_swa.hdf5'.format(self.model_name)))
        print('Logging Info - SWA model Saved')


# 周期学习率callback
class CyclicLR(Callback):
    def __init__(
            self,
            base_lr=0.001,
            max_lr=0.006,
            step_size=2000.,
            mode='triangular',
            gamma=1.,
            scale_fn=None,
            scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        if mode not in ['triangular', 'triangular2',
                        'exp_range']:
            raise KeyError("mode must be one of 'triangular', "
                           "'triangular2', or 'exp_range'")
        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn is None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1 / (2. ** (x - 1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma ** x
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr is not None:
            self.base_lr = new_base_lr
        if new_max_lr is not None:
            self.max_lr = new_max_lr
        if new_step_size is not None:
            self.step_size = new_step_size
        self.clr_iterations = 0.

    def clr(self):
        cycle = np.floor(1 + self.clr_iterations / (2 * self.step_size))
        x = np.abs(self.clr_iterations / self.step_size - 2 * cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr - self.base_lr) * \
                   np.maximum(0, (1 - x)) * self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr - self.base_lr) * \
                   np.maximum(0, (1 - x)) * self.scale_fn(self.clr_iterations)

    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())

    def on_batch_end(self, epoch, logs=None):

        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1
        K.set_value(self.model.optimizer.lr, self.clr())

        self.history.setdefault(
            'lr', []).append(
            K.get_value(
                self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)

# 通过转入列表获得所有手工特征的input
def get_dense_feature_inputs(dense_features_: list):
    dense_input = OrderedDict()
    for feature in dense_features_:
        dense_input[feature] = Input(shape=(1,), name=feature + '_input')
    return dense_input

# 通过传入input列表获得所有的fc的输出
def get_dense_feature_fc_list(dense_input_: OrderedDict, fc_dim=8, use_bias=False, l2_reg=1e-5):
    dense_input = list(dense_input_.values())
    fc_out_list = list(map(Dense(fc_dim, use_bias=use_bias, kernel_regularizer=l2(l2_reg)), dense_input))
    return fc_out_list

# 构建模型
def build_model(lstm_dim=64, emb_mat=None):
    print('Build model...')

    query_input = Input(shape=(max_seq_len,))
    title_input = Input(shape=(max_seq_len,))

    embedding = Embedding(emb_mat.shape[0], W2V_DIM, weights=[emb_mat], trainable=False, mask_zero=True)

    query_emb = embedding(query_input)
    query_emb = Dropout(0.2)(query_emb)

    title_emb = embedding(title_input)
    title_emb = Dropout(0.2)(title_emb)

    bilstm_1 = LSTM(units=lstm_dim, return_sequences=True)

    query_hidden = bilstm_1(query_emb)
    title_hidden = bilstm_1(title_emb)

    query_attend, title_attend = DotProductAttention()([query_hidden, title_hidden])

    query_enhance = concatenate([query_hidden, query_attend, subtract([query_hidden, query_attend]),
                                 multiply([query_hidden, query_attend])])  # [?,25,256]

    title_enhance = concatenate([title_hidden, title_attend,
                                 subtract([title_hidden, title_attend]),
                                 multiply([title_hidden, title_attend])])  # [?,25,256]
                                 
    feed_forward = TimeDistributed(Dense(units=lstm_dim, activation='relu'))

    bilstm_2 = LSTM(units=lstm_dim, return_sequences=True)

    query_compose = bilstm_2(feed_forward(query_enhance))  # [?,25,32]
    title_compose = bilstm_2(feed_forward(title_enhance))

    global_max_pooling = Lambda(lambda x: K.max(x, axis=1))
    query_avg = GlobalAveragePooling1D()(query_compose)
    query_max = global_max_pooling(query_compose)
    title_avg = GlobalAveragePooling1D()(title_compose)
    title_max = global_max_pooling(title_compose)

    inference_compose = concatenate([query_avg, query_max, title_avg, title_max])

    dense = Dense(units=lstm_dim, activation='tanh')(inference_compose)

    output = Dense(1, activation='sigmoid')(dense)
    model = Model(inputs=[query_input, title_input], outputs=output)
    return model


def auc(y_true, y_pred):
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)


from joblib import Parallel, delayed
from sklearn.metrics import roc_auc_score


def cal_AUC(labels, prob):
    try:
        return roc_auc_score(labels, prob)
    except:
        return 0.5


# 计算各个组的qauc值
def sum_AUC(mycombinedata):
    grouplist = mycombinedata[0]
    y_true = mycombinedata[1]
    y_pred = mycombinedata[2]

    if len(y_true) != sum(grouplist):
        print("评分函数中len(y_true)!=sum(group)")
        return
    start = 0
    sum_AUC = 0
    for group in grouplist:
        roc_auc = cal_AUC(y_true[start:start + group], y_pred[start:start + group])
        start = start + group
        sum_AUC = sum_AUC + roc_auc
    return sum_AUC

# 并行计算QAUC的值
def QAUC_parallel(y_true, y_pred, group):
    groupnum = 4
    import math
    group_len = math.ceil(len(group) / groupnum)
    groups = [group[i * group_len:(i + 1) * group_len] for i in range(groupnum)]
    mycombines = []

    start = 0
    for agroup in groups:
        mycombinedata = []
        mycombinedata.append(agroup)
        mycombinedata.append(y_true[start:start + sum(agroup)])
        mycombinedata.append(y_pred[start:start + sum(agroup)])
        start = start + sum(agroup)
        mycombines.append(mycombinedata)

    sum_AUC_ = Parallel(n_jobs=groupnum)(delayed(sum_AUC)(mycombinedata) for mycombinedata in mycombines)

    return sum(sum_AUC_) / len(group)


class Evaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()
        self.interval = interval
        self.x_val, self.val_group, self.y_val = validation_data
        self.best_score = 0.
        self.best_epoch = 0
        self.best_auc = 0.0
        self.best_auc_epoch = 0
        self.auc_list = []
        self.q_auc_list = []

    def on_epoch_end(self, epoch, log={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.x_val, verbose=0, batch_size=batch_size)
            auc_score = roc_auc_score(self.y_val, y_pred)
            self.auc_list.append(auc_score)
            if auc_score > self.best_auc:
                self.best_auc = auc_score
                self.best_auc_epoch = epoch + 1

            score_parallel = QAUC_parallel(self.y_val, y_pred, self.val_group)
            self.q_auc_list.append(score_parallel)
            if score_parallel > self.best_score:
                self.best_score = score_parallel
                self.best_epoch = epoch + 1

            print('\n ROC_AUC - epoch:%d - score:%.6f' % (epoch + 1, auc_score))
            print('\n Q_AUC - epoch:%d - score:%.6f' % (epoch + 1, score_parallel))

    def get_best_score_epoch(self):
        return self.best_score, self.best_epoch

    def get_best_auc_score_epoch(self):
        return self.best_auc, self.best_auc_epoch

    def show_result_list(self):
        print('auc', self.auc_list)
        print('\n')
        print('qauc', self.q_auc_list)


def seq_padding(X, max_len=20):
    return [x + [PAD] * (max_len - len(x)) if len(x) < max_len else x[:max_len] for x in X]


class DataGenerator(keras.utils.Sequence):

    def __init__(self, word2id, text_data, batch_size=1024 * 5):
        self.batch_size = batch_size
        self.word2id = word2id

        self.text_data = text_data

    def __len__(self):
        # 计算每一个epoch的迭代次数
        return math.ceil(len(self.text_data) / float(self.batch_size))

    def __getitem__(self, index):
        start = index * self.batch_size
        stop = (index + 1) * self.batch_size

        batch_text_df = self.text_data.iloc[start:stop]
        y = batch_text_df['label'].values

        Q = []
        D = []
        for query in batch_text_df['query']:
            query = query.split()
            Q.append([word2id.get(w, UNK) for w in query])

        for title in batch_text_df['title']:
            title = title.split()
            D.append([word2id.get(w, UNK) for w in title])

        Q_pad = seq_padding(Q, max_seq_len)
        D_pad = seq_padding(D, max_seq_len)
        return [np.array(Q_pad), np.array(D_pad)], y


flag = 'train'
batch_size = 1024 * 5

PAD = 0
UNK = 1

if __name__ == "__main__":
    if flag == 'train':
        from keras.models import load_model

        print('train')
        train_size = 98000000
        # 读取词向量矩阵
        emb_mat = np.load('/home/kesci/work/sunrui/NN_second_2e/word2vec_fasttext_6kw_nn_sim.npy')
        
        # 读取文本数据
        text_data = pd.read_csv('/home/kesci/input/bytedance/train_final.csv', usecols=[0, 1, 3, 4],
                                names=['query_id', 'query', 'title', 'label'], skiprows=range(0, 100000000),
                                nrows=100000000)
        print(f'shape ={text_data.shape}')
        # 读取 lgb feature 完毕
        
        # 读取词表
        with open('/home/kesci/work/sunrui/NN_second_2e/second_6kw_nn_sim.pkl', 'rb') as f:
            word2id = pickle.load(f)

        val_text_data = text_data[train_size:]
        train_text_data = text_data[:train_size]

        Q_val = []
        D_val = []
        for query in val_text_data['query']:
            query = query.split()
            Q_val.append([word2id.get(w, UNK) for w in query])  # 没有命中就返回UNK

        for title in val_text_data['title']:
            title = title.split()
            D_val.append([word2id.get(w, UNK) for w in title])

        val_query_input = seq_padding(Q_val, max_seq_len)
        val_title_input = seq_padding(D_val, max_seq_len)

        Y_val = val_text_data['label'].values

        train_generator = DataGenerator(word2id=word2id, text_data=train_text_data, batch_size=batch_size)

        val_text_data['query_id_nums'] = val_text_data.groupby(['query_id'])['label'].transform('count')

        val_group_df = val_text_data[['query_id', 'query_id_nums']].drop_duplicates()
        val_group = val_group_df.query_id_nums.get_values()

        swa = SWA(checkpoint_dir='./sunrui/swa/', model_name='swa.model')

        clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                       step_size=2 * train_size // batch_size, mode='triangular')

        model = build_model(64, emb_mat)
        print('load end')
        filepath = "./sunrui/nn/weights-{epoch:02d}_esim_64_nn_sim.hdf5"
        checkpoint = ModelCheckpoint(filepath, verbose=1)

        early_stopping = EarlyStopping(monitor='val_auc', patience=5, verbose=1, mode='max')

        val_model_input = [val_query_input, val_title_input]

        eval_callback = Evaluation(
            validation_data=(
                val_model_input, val_group, Y_val))

        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[auc])
        model.summary()
        model.fit_generator(train_generator, epochs=epochs,
                            validation_data=(val_model_input, Y_val),
                            callbacks=[early_stopping, eval_callback, swa, clr, checkpoint], shuffle=True, workers=8,
                            use_multiprocessing=True)
    else:
        from keras.models import load_model

        """
            在这里修改最好epoch的轮次模型名称和提交的结果文件名
        """
        model_file_name = './sunrui/nn/weights-04_esim_64_single_pure_text.hdf5'
        result_file_name = './sunrui/esim_64_single_pure_textesim_64_single_pure_text_epoch4_second_20kw.csv'

        model = load_model(model_file_name, custom_objects={
            'DotProductAttention': DotProductAttention,
            'auc': auc
        })

        test = pd.read_csv('/home/kesci/input/bytedance/test_final_part1.csv', header=None)
        test.columns = ['query_id', 'query', 'query_title_id', 'title']

        with open('./sunrui/words2id_all.pkl', 'rb') as f:
            word2id = pickle.load(f)

        Q_val = []
        D_val = []
        for query in test['query']:
            query = query.split()
            Q_val.append([word2id[w] for w in query])  # 没有命中就返回UNK

        for title in test['title']:
            title = title.split()
            D_val.append([word2id[w] for w in title])

        test_query_input = seq_padding(Q_val, max_seq_len)
        test_title_input = seq_padding(D_val, max_seq_len)

        import pandas as pd

        preds = model.predict([test_query_input, test_title_input], batch_size=batch_size)

        preds_np = np.squeeze(preds)

        test = pd.read_csv('/home/kesci/input/bytedance/test_final_part1.csv', header=None)
        test.columns = ['query_id', 'query', 'query_title_id', 'title']

        test = test.drop(['query', 'title'], axis=1)

        test_y = {'prediction': preds_np}
        test_y = pd.DataFrame(test_y)

        test_result = pd.concat([test, test_y], axis=1)

        test_result.to_csv(result_file_name, index=None, header=None)

In [ ]:
# 以下是带有特征的esim网络的训练 前1e数据的 隐层个数为128 我们选择了前2轮的模型用于模型融合
# 下面几个ceil都是相同的代码 只不过读取的是不同的数据 

In [ ]:
import math
import os
import pickle
from collections import OrderedDict

import numpy as np
import pandas as pd
import tensorflow as tf
from keras import Input, Model
from keras.backend.tensorflow_backend import set_session
from keras.callbacks import EarlyStopping
from keras.layers import Embedding, Dense, Dropout, Lambda, concatenate, GlobalAveragePooling1D, subtract, multiply, \
    TimeDistributed, LSTM
from keras.regularizers import l2
from tensorflow import set_random_seed

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
set_session(sess)

seed = 2019

set_random_seed(seed)  # Tensorflow
np.random.seed(seed)  # NumPy

from keras.callbacks import Callback, ModelCheckpoint
import keras
import keras.backend as K
from keras.engine import Layer

import numpy as np

import logging

logger = logging.getLogger(__name__)
logger.setLevel(level=logging.INFO)
handler = logging.FileHandler("./sunrui/gated_esim_64_pre_1e_64_log.txt")  # 训练前1e的数据
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

console = logging.StreamHandler()
console.setLevel(logging.INFO)

logger.addHandler(handler)
logger.addHandler(console)


class DotProductAttention(Layer):
    def __init__(self, return_attend_weight=False, keep_mask=True, **kwargs):
        self.return_attend_weight = return_attend_weight
        self.keep_mask = keep_mask
        self.supports_masking = True
        super(DotProductAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        input_shape_a, input_shape_b = input_shape

        if len(input_shape_a) != 3 or len(input_shape_b) != 3:
            raise ValueError('Inputs into DotProductAttention should be 3D tensors')

        if input_shape_a[-1] != input_shape_b[-1]:
            raise ValueError('Inputs into DotProductAttention should have the same dimensionality at the last axis')

    def call(self, inputs, mask=None):
        assert isinstance(inputs, list)
        inputs_a, inputs_b = inputs

        if mask is not None:
            mask_a, mask_b = mask
        else:
            mask_a, mask_b = None, None

        e = K.exp(K.batch_dot(inputs_a, inputs_b, axes=2))  # similarity between a & b

        if mask_a is not None:
            e *= K.expand_dims(K.cast(mask_a, K.floatx()), 2)
        if mask_b is not None:
            e *= K.expand_dims(K.cast(mask_b, K.floatx()), 1)

        e_b = e / K.cast(K.sum(e, axis=2, keepdims=True) + K.epsilon(), K.floatx())  # attention weight over b
        e_a = e / K.cast(K.sum(e, axis=1, keepdims=True) + K.epsilon(), K.floatx())  # attention weight over a

        if self.return_attend_weight:
            return [e_b, e_a]

        a_attend = K.batch_dot(e_b, inputs_b, axes=(2, 1))  # a attend to b
        b_attend = K.batch_dot(e_a, inputs_a, axes=(1, 1))  # b attend to a
        return [a_attend, b_attend]

    def compute_mask(self, inputs, mask=None):
        if self.keep_mask:
            return mask
        else:
            return [None, None]

    def compute_output_shape(self, input_shape):
        if self.return_attend_weight:
            input_shape_a, input_shape_b = input_shape
            return [(input_shape_a[0], input_shape_a[1], input_shape_b[1]),
                    (input_shape_a[0], input_shape_a[1], input_shape_b[1])]
        return input_shape


class SWA(Callback):
    def __init__(self, checkpoint_dir, model_name, swa_start=1):
        super(SWA, self).__init__()
        self.checkpoint_dir = checkpoint_dir
        self.model_name = model_name
        self.swa_start = swa_start
        self.swa_model = None  # the model that we will use to store the average of the weights once SWA begins

    def on_train_begin(self, logs=None):
        self.epoch = 0
        self.swa_n = 0
        # self.swa_model = copy.deepcopy(self.model)  # make a copy of the model we're training
        # Note: I found deep copy of a model with customized layer would give errors
        self.swa_model = keras.models.clone_model(self.model)
        self.swa_model.set_weights(self.model.get_weights())  # see: https://github.com/keras-team/keras/issues/1765

    def on_epoch_end(self, epoch, logs=None):
        if (self.epoch + 1) >= self.swa_start:
            self.update_average_model()
            self.swa_n += 1

        self.epoch += 1

    def update_average_model(self):
        # update running average of parameters
        alpha = 1. / (self.swa_n + 1)
        for layer, swa_layer in zip(self.model.layers, self.swa_model.layers):
            weights = []
            for w1, w2 in zip(swa_layer.get_weights(), layer.get_weights()):
                weights.append((1 - alpha) * w1 + alpha * w2)
            swa_layer.set_weights(weights)

    def on_train_end(self, logs=None):
        print('Logging Info - Saving SWA model checkpoint: %s_swa.hdf5\n' % self.model_name)
        self.swa_model.save_weights(os.path.join(self.checkpoint_dir, '{}_swa.hdf5'.format(self.model_name)))
        print('Logging Info - SWA model Saved')


class CyclicLR(Callback):
    def __init__(
            self,
            base_lr=0.001,
            max_lr=0.006,
            step_size=2000.,
            mode='triangular',
            gamma=1.,
            scale_fn=None,
            scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        if mode not in ['triangular', 'triangular2',
                        'exp_range']:
            raise KeyError("mode must be one of 'triangular', "
                           "'triangular2', or 'exp_range'")
        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn is None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1 / (2. ** (x - 1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma ** x
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr is not None:
            self.base_lr = new_base_lr
        if new_max_lr is not None:
            self.max_lr = new_max_lr
        if new_step_size is not None:
            self.step_size = new_step_size
        self.clr_iterations = 0.

    def clr(self):
        cycle = np.floor(1 + self.clr_iterations / (2 * self.step_size))
        x = np.abs(self.clr_iterations / self.step_size - 2 * cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr - self.base_lr) * \
                   np.maximum(0, (1 - x)) * self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr - self.base_lr) * \
                   np.maximum(0, (1 - x)) * self.scale_fn(self.clr_iterations)

    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())

    def on_batch_end(self, epoch, logs=None):

        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1
        K.set_value(self.model.optimizer.lr, self.clr())

        self.history.setdefault(
            'lr', []).append(
            K.get_value(
                self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)


def get_dense_feature_inputs(dense_features_: list):
    dense_input = OrderedDict()
    for feature in dense_features_:
        dense_input[feature] = Input(shape=(1,), name=feature + '_input')
    return dense_input


def get_dense_feature_fc_list(dense_input_: OrderedDict, fc_dim=8, use_bias=True, l2_reg=1e-4):
    dense_input = list(dense_input_.values())
    fc_out_list = list(map(Dense(fc_dim, use_bias=use_bias, kernel_regularizer=l2(l2_reg)), dense_input))
    return fc_out_list


def build_model(lstm_dim=64, emb_mat=None):
    print('Build model...')

    query_input = Input(shape=(max_seq_len,))
    title_input = Input(shape=(max_seq_len,))

    embedding = Embedding(emb_mat.shape[0], W2V_DIM, weights=[emb_mat], trainable=False, mask_zero=True)

    query_emb = embedding(query_input)
    query_emb = Dropout(0.2)(query_emb)

    title_emb = embedding(title_input)
    title_emb = Dropout(0.2)(title_emb)

    bilstm_1 = LSTM(units=lstm_dim, return_sequences=True)

    query_hidden = bilstm_1(query_emb)
    title_hidden = bilstm_1(title_emb)

    query_attend, title_attend = DotProductAttention()([query_hidden, title_hidden])

    query_enhance = concatenate([query_hidden, query_attend, subtract([query_hidden, query_attend]),
                                 multiply([query_hidden, query_attend])])  # [?,25,256]

    title_enhance = concatenate([title_hidden, title_attend,
                                 subtract([title_hidden, title_attend]),
                                 multiply([title_hidden, title_attend])])  # [?,25,256]

    # inference composition
    feed_forward = TimeDistributed(Dense(units=lstm_dim, activation='relu'))

    bilstm_2 = LSTM(units=lstm_dim, return_sequences=True)

    query_compose = bilstm_2(feed_forward(query_enhance))  # [?,25,32]
    title_compose = bilstm_2(feed_forward(title_enhance))

    global_max_pooling = Lambda(lambda x: K.max(x, axis=1))  # GlobalMaxPooling1D didn't support masking
    query_avg = GlobalAveragePooling1D()(query_compose)
    query_max = global_max_pooling(query_compose)
    title_avg = GlobalAveragePooling1D()(title_compose)
    title_max = global_max_pooling(title_compose)

    lgb_dense_feature_input = get_dense_feature_inputs(used_lgb_dense_feature)

    dense_fc_list = get_dense_feature_fc_list(lgb_dense_feature_input)

    if len(dense_fc_list) > 1:
        dense_feature_concat = concatenate(dense_fc_list)
    else:
        dense_feature_concat = dense_fc_list[0]

    inference_compose = concatenate([query_avg, query_max, title_avg, title_max])

    # inference_compose = BatchNormalization()(inference_compose)  # 尝试

    dense_esim = Dense(units=lstm_dim)(inference_compose)

    dense_feature_gate = Dense(lstm_dim, activation='sigmoid')(dense_feature_concat)

    gated_esim = Lambda(lambda x: x[0] * x[1])([dense_esim, dense_feature_gate])

    gated_esim = Dense(lstm_dim, activation='elu')(gated_esim)
    model_dense_input = [lgb_dense_feature_input[feat] for feat in used_lgb_dense_feature]

    # gated_esim = BatchNormalization()(gated_esim)  # 加了BN 训练不稳定 val loss会跳 但是性能尚可
    # gated_esim = Dropout(0.1)(gated_esim)

    output = Dense(1, activation='sigmoid')(gated_esim)
    model = Model(inputs=[query_input, title_input] + model_dense_input, outputs=output)
    return model


def auc(y_true, y_pred):
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)


from joblib import Parallel, delayed
from sklearn.metrics import roc_auc_score


def cal_AUC(labels, prob):
    try:
        return roc_auc_score(labels, prob)
    except:
        return 0.5


# 计算各个组的qauc值
def sum_AUC(mycombinedata):
    grouplist = mycombinedata[0]
    y_true = mycombinedata[1]
    y_pred = mycombinedata[2]

    if len(y_true) != sum(grouplist):
        print("评分函数中len(y_true)!=sum(group)")
        return
    start = 0
    sum_AUC = 0
    for group in grouplist:
        roc_auc = cal_AUC(y_true[start:start + group], y_pred[start:start + group])
        start = start + group
        sum_AUC = sum_AUC + roc_auc
    return sum_AUC


def QAUC_parallel(y_true, y_pred, group):
    groupnum = 4
    import math
    group_len = math.ceil(len(group) / groupnum)
    groups = [group[i * group_len:(i + 1) * group_len] for i in range(groupnum)]
    mycombines = []

    start = 0
    for agroup in groups:
        mycombinedata = []
        mycombinedata.append(agroup)
        mycombinedata.append(y_true[start:start + sum(agroup)])
        mycombinedata.append(y_pred[start:start + sum(agroup)])
        start = start + sum(agroup)
        mycombines.append(mycombinedata)

    sum_AUC_ = Parallel(n_jobs=groupnum)(delayed(sum_AUC)(mycombinedata) for mycombinedata in mycombines)

    return sum(sum_AUC_) / len(group)


class Evaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()
        self.interval = interval
        self.x_val, self.val_group, self.y_val = validation_data
        self.best_score = 0.
        self.best_epoch = 0
        self.best_auc = 0.0
        self.best_auc_epoch = 0
        self.auc_list = []
        self.q_auc_list = []

    def on_epoch_end(self, epoch, log={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.x_val, verbose=0, batch_size=batch_size)
            auc_score = roc_auc_score(self.y_val, y_pred)
            self.auc_list.append(auc_score)
            if auc_score > self.best_auc:
                self.best_auc = auc_score
                self.best_auc_epoch = epoch + 1

            score_parallel = QAUC_parallel(self.y_val, y_pred, self.val_group)
            self.q_auc_list.append(score_parallel)
            if score_parallel > self.best_score:
                self.best_score = score_parallel
                self.best_epoch = epoch + 1
            logger.info(f'Q_AUC = {score_parallel} epoch = {epoch + 1}')

            print('\n ROC_AUC - epoch:%d - score:%.6f' % (epoch + 1, auc_score))
            print('\n Q_AUC - epoch:%d - score:%.6f' % (epoch + 1, score_parallel))

    def get_best_score_epoch(self):
        return self.best_score, self.best_epoch

    def get_best_auc_score_epoch(self):
        return self.best_auc, self.best_auc_epoch

    def show_result_list(self):
        print('auc', self.auc_list)
        print('\n')
        print('qauc', self.q_auc_list)


def seq_padding(X, max_len):
    return [x + [PAD] * (max_len - len(x)) if len(x) < max_len else x[:max_len] for x in X]


class DataGenerator(keras.utils.Sequence):

    def __init__(self, word2id, text_data, lgb_data, batch_size=1024 * 5):
        self.batch_size = batch_size
        self.word2id = word2id

        self.text_data = text_data
        self.lgb_data = lgb_data

    def __len__(self):
        # 计算每一个epoch的迭代次数
        return math.ceil(len(self.text_data) / float(self.batch_size))

    def __getitem__(self, index):
        start = index * self.batch_size
        stop = (index + 1) * self.batch_size

        batch_lgb_df = self.lgb_data.iloc[start:stop]
        batch_text_df = self.text_data.iloc[start:stop]
        y = batch_text_df['label'].values

        train_lgb_input = [batch_lgb_df[feat].values for feat in used_lgb_dense_feature]

        Q = []
        D = []
        for query in batch_text_df['query']:
            query = query.split()
            Q.append([word2id[w] for w in query])

        for title in batch_text_df['title']:
            title = title.split()
            D.append([word2id[w] for w in title])

        Q_pad = seq_padding(Q, max_seq_len)
        D_pad = seq_padding(D, max_seq_len)
        return [np.array(Q_pad), np.array(D_pad)] + train_lgb_input, y


flag = 'train'
batch_size = 1024 * 5

PAD = 0
UNK = 1

W2V_DIM = 200

max_seq_len = 25
epochs = 20


def get_used_feature_names(featurecol_h5):
    features = []
    for k, v in featurecol_h5.items():
        features.extend(v)
    return features


featurecol_h5 = {
    'sim_feat': ['jaccard_q3_t3',
                 'jaccard_q3_t5',
                 'jaccard_q5_t5', 'levenshtein_q5_t5',
                 'jaccard_q5_t10', 'levenshtein_q5_t10',
                 'jaccard_q10_t10', 'levenshtein_q10_t10',
                 'jaccard_q15_t25', 'levenshtein_q15_t25',
                 'jaccard', 'levenshtein'],

    'len_feat': ["querykw_num", "titlekw_num"],

    "title_nunique_query": ["title_nunique_query"],
    "query_nunique_title": ["query_nunique_title"],

    'title_score_count_feat': ["title_score_count", "title_score_click_num"],
    'title_code_score_feat': ["title_code_score"],
    'title_convert_feat': ["title_code_convert", 'title_code_label_count'],

    'query_count': ["query_code_count"],
    'title_count': ["title_count"],

    "match_feat": ['count_match', 'blockcount_match', 'proximity', 'maxMatchBlockLen',
                   'q1_match_start', 'q1_match_end'],

    "BM25": ["BM25"],
}

othercols = ["titlekw_querykw_diff", "titlekw_querykw_rate"]


def reduce_mem_usage(D, verbose=True):
    start_mem = D.memory_usage().sum() / 1024 ** 2
    for c, d in zip(D.columns, D.dtypes):
        if d.kind == 'f':
            D[c] = pd.to_numeric(D[c], downcast='float')
        elif d.kind == 'i':
            D[c] = pd.to_numeric(D[c], downcast='signed')
    end_mem = D.memory_usage().sum() / 1024 ** 2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (
            start_mem - end_mem) / start_mem))
    return D


def ReadData(datatype='train', nrows=1000000000):
    if datatype == 'train':
        id_feature = '/home/kesci/input/bytedance/train_final.csv'
        usecols = [0, 4]
        names = ['query_id', 'label']

        print("read ", id_feature)
        DataSet = pd.read_csv(id_feature,
                              header=None,
                              nrows=nrows,
                              usecols=usecols,
                              names=names
                              )
        path_h5 = "/home/kesci/work/pre_3billion_data/train/"
    elif datatype == 'test1':
        id_feature = '/home/kesci/input/bytedance/test_final_part1.csv'
        usecols = [0, 2]
        names = ['query_id', 'query_title_id']
        path_h5 = "/home/kesci/work/post_4kw_data/test1/"
        print("read ", id_feature)
        DataSet = pd.read_csv(id_feature,
                              header=None,
                              nrows=nrows,
                              usecols=usecols,
                              names=names
                              )
    elif datatype == 'test2':
        id_feature = '/home/kesci/input/bytedance/bytedance_contest.final_2.csv'
        usecols = [0, 2]
        names = ['query_id', 'query_title_id']
        path_h5 = "/home/kesci/work/post_4kw_data/test2/"
        print("read ", id_feature)
        DataSet = pd.read_csv(id_feature,
                              header=None,
                              nrows=nrows,
                              usecols=usecols,
                              names=names
                              )

    print("length:", DataSet.__len__())
    DataSet = reduce_mem_usage(DataSet, verbose=True)

    featuremap_h5 = {
        'cross_feat': path_h5 + f'cross_{datatype}_feat.h5',

        'query_pos_feat': path_h5 + f'query_pos_{datatype}_feat.h5',
        'title_pos_feat': path_h5 + f'title_pos_{datatype}_feat.h5',

        'match_feat': path_h5 + f'query_match_{datatype}_feat.h5',
        'editDistance_feat': path_h5 + f'editDistance_{datatype}_feat.h5',

        'sim_feat': path_h5 + f'sim_{datatype}_feat.h5',
        'tag_score_feat': path_h5 + f'tag_score_10foldtime_{datatype}_feat.h5',
        'title_score_count_feat': path_h5 + f'title_score_count_{datatype}_feat.h5',
        'title_code_score_feat': path_h5 + f'title_code_score_10foldtime_{datatype}_feat.h5',
        'title_convert_feat': path_h5 + f'title_convert_{datatype}.h5',
        'sif_feat': path_h5 + f'sif_{datatype}_post_4kw.h5',
        'len_feat': path_h5 + f'len_{datatype}_feat.h5',

        'title_count': path_h5 + f'count_feature_{datatype}.h5',
        'query_count': path_h5 + f'query_count_all_{datatype}.h5',

        "title_nunique_query": path_h5 + f'nunique_feature_{datatype}.h5',
        "query_nunique_title": path_h5 + f'query_nunique_title_all_{datatype}.h5',

        'tag': path_h5 + f'tag_{datatype}.h5',
        "tag_convert_feat": path_h5 + f"tag_convert_{datatype}.h5",
        "query_convert": path_h5 + f"query_convert_{datatype}.h5",

        "M_cosine": path_h5 + f"M_sim_{datatype}_feat.h5",
        "M_tfidf_cosine": path_h5 + f"M_tfidf_sim_{datatype}_feat.h5",
        "BM25": path_h5 + f'BM25_{datatype}_feat.h5',
        'NN_SIM': path_h5 + f'nn_sim_feature.h5',

        'editdistance_relativepos': path_h5 + f'editdistance_relativepos_{datatype}_feat.h5',
        'fuzz': path_h5 + f"fuzz_{datatype}_feat.h5",
        'textpair': path_h5 + f"textpair_{datatype}_feat.h5",

        'sen_dis': path_h5 + f"sen_dis_{datatype}_200.h5",
        'sen_dis2': path_h5 + f"sen_dis2_{datatype}_200.h5",
    }

    for featurefile in featurecol_h5:
        print("read ", featuremap_h5[featurefile])
        feature_set = pd.read_hdf(featuremap_h5[featurefile],
                                  key='data',
                                  start=0,
                                  stop=nrows)[featurecol_h5[featurefile]].reset_index(drop=True)
        print("length:", feature_set.__len__())
        # print(feature_set.head(1))
        # feature_set=reduce_mem_usage(feature_set, verbose=True)
        DataSet = pd.concat([DataSet, feature_set], axis=1)

    DataSet["titlekw_querykw_diff"] = DataSet["titlekw_num"] - DataSet["querykw_num"]
    DataSet["titlekw_querykw_rate"] = DataSet["titlekw_num"] / DataSet["querykw_num"]

    if "title_code_score" in DataSet.columns:
        DataSet.title_code_score = DataSet.title_code_score.fillna(0)
    if "tag_score" in DataSet.columns:
        DataSet.tag_score = DataSet.tag_score.fillna(0)

    DataSet = reduce_mem_usage(DataSet, verbose=True)
    print("Data Read Finish!")
    return DataSet


if __name__ == "__main__":
    if flag == 'train':

        train_size = 98000000
        emb_mat = np.load('/home/kesci/work/sunrui/NN_second_2e/word2vec_fasttext_6kw_nn_sim.npy')
        
        lgb_data = ReadData(datatype='train', nrows=100000000)
        used_lgb_dense_feature = get_used_feature_names(featurecol_h5) + othercols

        print(used_lgb_dense_feature)

        text_data = pd.read_csv('/home/kesci/input/bytedance/train_final.csv', usecols=[0, 1, 3, 4], header=None,
            names=['query_id', 'query', 'title', 'label'], nrows=100000000)
        # 读取 lgb feature
        print(text_data.shape)
        lgb_data[used_lgb_dense_feature] = lgb_data[used_lgb_dense_feature].fillna(-1, )

        train_lgb_data = lgb_data[:train_size]
        val_lgb_data = lgb_data[train_size:]
        # 读取 lgb feature 完毕

        with open('/home/kesci/work/sunrui/NN_second_2e/second_6kw_nn_sim.pkl', 'rb') as f:
            word2id = pickle.load(f)

        val_text_data = text_data[train_size:]
        train_text_data = text_data[:train_size]

        Q_val = []
        D_val = []
        for query in val_text_data['query']:
            query = query.split()
            Q_val.append([word2id[w] for w in query])  # 没有命中就返回UNK

        for title in val_text_data['title']:
            title = title.split()
            D_val.append([word2id[w] for w in title])

        val_query_input = seq_padding(Q_val, max_seq_len)
        val_title_input = seq_padding(D_val, max_seq_len)

        Y_val = val_text_data['label'].values

        train_generator = DataGenerator(word2id=word2id, text_data=train_text_data,
                                        lgb_data=train_lgb_data, batch_size=batch_size)

        val_text_data['query_id_nums'] = val_text_data.groupby(['query_id'])['label'].transform('count')

        val_group_df = val_text_data[['query_id', 'query_id_nums']].drop_duplicates()
        val_group = val_group_df.query_id_nums.get_values()

        swa = SWA(checkpoint_dir='./sunrui/swa/', model_name='swa.model')

        clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                       step_size=2 * train_size // batch_size, mode='triangular')

        model = build_model(lstm_dim=128, emb_mat=emb_mat)

        filepath = "/home/kesci/work/sunrui/nn/gated_pre_1e/128dim/gated-{epoch:02d}_esim_128_pre1e.hdf5"
        checkpoint = ModelCheckpoint(filepath, verbose=1)

        early_stopping = EarlyStopping(monitor='val_auc', patience=5, verbose=1, mode='max')

        val_lgb_input = [val_lgb_data[feat].values for feat in used_lgb_dense_feature]

        # train_model_input = [train_query_input, train_title_input] + train_lgb_input
        val_model_input = [val_query_input, val_title_input] + val_lgb_input

        eval_callback = Evaluation(
            validation_data=(
                val_model_input, val_group, Y_val))

        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[auc])
        model.summary()
        model.fit_generator(train_generator, epochs=epochs,
                            validation_data=(val_model_input, Y_val),
                            callbacks=[early_stopping, eval_callback, swa, clr, checkpoint], shuffle=True,
                            workers=2,
                            use_multiprocessing=True)
    else:
        pass

In [ ]:
# 以下是带有特征的esim网络 后1e数据的 隐层个数为64 我们选择了第5轮的结果
#（一开始训练了一轮 中途需要资源就停止了训练 读取再训练了4轮 所以是5轮的结果）

In [ ]:
import math
import os
import pickle
from collections import OrderedDict

import numpy as np
import pandas as pd
import tensorflow as tf
from keras import Input, Model
from keras.backend.tensorflow_backend import set_session
from keras.callbacks import EarlyStopping
from keras.layers import Embedding, Dense, Dropout, Lambda, concatenate, GlobalAveragePooling1D, subtract, multiply, \
    TimeDistributed, LSTM
from keras.regularizers import l2
from tensorflow import set_random_seed

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
set_session(sess)

seed = 2019

set_random_seed(seed)  # Tensorflow
np.random.seed(seed)  # NumPy

from keras.callbacks import Callback, ModelCheckpoint
import keras
import keras.backend as K
from keras.engine import Layer

import numpy as np

import logging

logger = logging.getLogger(__name__)
logger.setLevel(level=logging.INFO)
handler = logging.FileHandler("./sunrui/gated_esim_64_1e_log.txt")
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

console = logging.StreamHandler()
console.setLevel(logging.INFO)

logger.addHandler(handler)
logger.addHandler(console)


def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (
            start_mem - end_mem) / start_mem))
    return df


class DotProductAttention(Layer):
    """
    dot-product-attention mechanism, supporting masking
    """

    def __init__(self, return_attend_weight=False, keep_mask=True, **kwargs):
        self.return_attend_weight = return_attend_weight
        self.keep_mask = keep_mask
        self.supports_masking = True
        super(DotProductAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        input_shape_a, input_shape_b = input_shape

        if len(input_shape_a) != 3 or len(input_shape_b) != 3:
            raise ValueError('Inputs into DotProductAttention should be 3D tensors')

        if input_shape_a[-1] != input_shape_b[-1]:
            raise ValueError('Inputs into DotProductAttention should have the same dimensionality at the last axis')

    def call(self, inputs, mask=None):
        assert isinstance(inputs, list)
        inputs_a, inputs_b = inputs

        if mask is not None:
            mask_a, mask_b = mask
        else:
            mask_a, mask_b = None, None

        e = K.exp(K.batch_dot(inputs_a, inputs_b, axes=2))  # similarity between a & b

        # apply mask before normalization (softmax)
        if mask_a is not None:
            e *= K.expand_dims(K.cast(mask_a, K.floatx()), 2)
        if mask_b is not None:
            e *= K.expand_dims(K.cast(mask_b, K.floatx()), 1)

        e_b = e / K.cast(K.sum(e, axis=2, keepdims=True) + K.epsilon(), K.floatx())  # attention weight over b
        e_a = e / K.cast(K.sum(e, axis=1, keepdims=True) + K.epsilon(), K.floatx())  # attention weight over a

        if self.return_attend_weight:
            return [e_b, e_a]

        a_attend = K.batch_dot(e_b, inputs_b, axes=(2, 1))  # a attend to b
        b_attend = K.batch_dot(e_a, inputs_a, axes=(1, 1))  # b attend to a
        return [a_attend, b_attend]

    def compute_mask(self, inputs, mask=None):
        if self.keep_mask:
            return mask
        else:
            return [None, None]

    def compute_output_shape(self, input_shape):
        if self.return_attend_weight:
            input_shape_a, input_shape_b = input_shape
            return [(input_shape_a[0], input_shape_a[1], input_shape_b[1]),
                    (input_shape_a[0], input_shape_a[1], input_shape_b[1])]
        return input_shape


class SWA(Callback):
    def __init__(self, checkpoint_dir, model_name, swa_start=1):
        """
        :param checkpoint_dir: the directory where the model will be saved in
        :param model_name: the name of model we're training
        :param swa_start: the epoch when averaging begins. We generally pre-train the network for a certain amount of
                          epochs to start (swa_start > 1), as opposed to starting to track the average from the
                          very beginning.
        """
        super(SWA, self).__init__()
        self.checkpoint_dir = checkpoint_dir
        self.model_name = model_name
        self.swa_start = swa_start
        self.swa_model = None  # the model that we will use to store the average of the weights once SWA begins

    def on_train_begin(self, logs=None):
        self.epoch = 0
        self.swa_n = 0
        # self.swa_model = copy.deepcopy(self.model)  # make a copy of the model we're training
        # Note: I found deep copy of a model with customized layer would give errors
        self.swa_model = keras.models.clone_model(self.model)
        self.swa_model.set_weights(self.model.get_weights())  # see: https://github.com/keras-team/keras/issues/1765

    def on_epoch_end(self, epoch, logs=None):
        if (self.epoch + 1) >= self.swa_start:
            self.update_average_model()
            self.swa_n += 1

        self.epoch += 1

    def update_average_model(self):
        # update running average of parameters
        alpha = 1. / (self.swa_n + 1)
        for layer, swa_layer in zip(self.model.layers, self.swa_model.layers):
            weights = []
            for w1, w2 in zip(swa_layer.get_weights(), layer.get_weights()):
                weights.append((1 - alpha) * w1 + alpha * w2)
            swa_layer.set_weights(weights)

    def on_train_end(self, logs=None):
        print('Logging Info - Saving SWA model checkpoint: %s_swa.hdf5\n' % self.model_name)
        self.swa_model.save_weights(os.path.join(self.checkpoint_dir, '{}_swa.hdf5'.format(self.model_name)))
        print('Logging Info - SWA model Saved')


class CyclicLR(Callback):
    def __init__(
            self,
            base_lr=0.001,
            max_lr=0.006,
            step_size=2000.,
            mode='triangular',
            gamma=1.,
            scale_fn=None,
            scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        if mode not in ['triangular', 'triangular2',
                        'exp_range']:
            raise KeyError("mode must be one of 'triangular', "
                           "'triangular2', or 'exp_range'")
        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn is None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1 / (2. ** (x - 1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma ** x
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr is not None:
            self.base_lr = new_base_lr
        if new_max_lr is not None:
            self.max_lr = new_max_lr
        if new_step_size is not None:
            self.step_size = new_step_size
        self.clr_iterations = 0.

    def clr(self):
        cycle = np.floor(1 + self.clr_iterations / (2 * self.step_size))
        x = np.abs(self.clr_iterations / self.step_size - 2 * cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr - self.base_lr) * \
                   np.maximum(0, (1 - x)) * self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr - self.base_lr) * \
                   np.maximum(0, (1 - x)) * self.scale_fn(self.clr_iterations)

    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())

    def on_batch_end(self, epoch, logs=None):

        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1
        K.set_value(self.model.optimizer.lr, self.clr())

        self.history.setdefault(
            'lr', []).append(
            K.get_value(
                self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)


def get_dense_feature_inputs(dense_features_: list):
    dense_input = OrderedDict()
    for feature in dense_features_:
        dense_input[feature] = Input(shape=(1,), name=feature + '_input')
    return dense_input


def get_dense_feature_fc_list(dense_input_: OrderedDict, fc_dim=8, use_bias=True, l2_reg=1e-4):
    dense_input = list(dense_input_.values())
    fc_out_list = list(map(Dense(fc_dim, use_bias=use_bias, kernel_regularizer=l2(l2_reg)), dense_input))
    return fc_out_list


def build_model(lstm_dim=64, emb_mat=None):
    print('Build model...')

    query_input = Input(shape=(max_seq_len,))
    title_input = Input(shape=(max_seq_len,))

    embedding = Embedding(emb_mat.shape[0], W2V_DIM, weights=[emb_mat], trainable=False, mask_zero=True)

    query_emb = embedding(query_input)
    query_emb = Dropout(0.2)(query_emb)

    title_emb = embedding(title_input)
    title_emb = Dropout(0.2)(title_emb)

    bilstm_1 = LSTM(units=lstm_dim, return_sequences=True)

    query_hidden = bilstm_1(query_emb)
    title_hidden = bilstm_1(title_emb)

    query_attend, title_attend = DotProductAttention()([query_hidden, title_hidden])

    query_enhance = concatenate([query_hidden, query_attend, subtract([query_hidden, query_attend]),
                                 multiply([query_hidden, query_attend])])  # [?,25,256]

    title_enhance = concatenate([title_hidden, title_attend,
                                 subtract([title_hidden, title_attend]),
                                 multiply([title_hidden, title_attend])])  # [?,25,256]

    # inference composition
    feed_forward = TimeDistributed(Dense(units=lstm_dim, activation='relu'))

    bilstm_2 = LSTM(units=lstm_dim, return_sequences=True)

    query_compose = bilstm_2(feed_forward(query_enhance))  # [?,25,32]
    title_compose = bilstm_2(feed_forward(title_enhance))

    global_max_pooling = Lambda(lambda x: K.max(x, axis=1))  # GlobalMaxPooling1D didn't support masking
    query_avg = GlobalAveragePooling1D()(query_compose)
    query_max = global_max_pooling(query_compose)
    title_avg = GlobalAveragePooling1D()(title_compose)
    title_max = global_max_pooling(title_compose)

    lgb_dense_feature_input = get_dense_feature_inputs(used_lgb_dense_feature)

    dense_fc_list = get_dense_feature_fc_list(lgb_dense_feature_input)

    if len(dense_fc_list) > 1:
        dense_feature_concat = concatenate(dense_fc_list)
    else:
        dense_feature_concat = dense_fc_list[0]

    inference_compose = concatenate([query_avg, query_max, title_avg, title_max])

    # inference_compose = BatchNormalization()(inference_compose)  # 尝试

    dense_esim = Dense(units=lstm_dim)(inference_compose)

    dense_feature_gate = Dense(lstm_dim, activation='sigmoid')(dense_feature_concat)

    gated_esim = Lambda(lambda x: x[0] * x[1])([dense_esim, dense_feature_gate])

    gated_esim = Dense(lstm_dim, activation='elu')(gated_esim)
    model_dense_input = [lgb_dense_feature_input[feat] for feat in used_lgb_dense_feature]

    # gated_esim = BatchNormalization()(gated_esim)  # 加了BN 训练不稳定 val loss会跳 但是性能尚可
    # gated_esim = Dropout(0.1)(gated_esim)

    output = Dense(1, activation='sigmoid')(gated_esim)
    model = Model(inputs=[query_input, title_input] + model_dense_input, outputs=output)
    return model


def auc(y_true, y_pred):
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)


from joblib import Parallel, delayed
from sklearn.metrics import roc_auc_score


def cal_AUC(labels, prob):
    try:
        return roc_auc_score(labels, prob)
    except:
        return 0.5


# 计算各个组的qauc值
def sum_AUC(mycombinedata):
    grouplist = mycombinedata[0]
    y_true = mycombinedata[1]
    y_pred = mycombinedata[2]

    if len(y_true) != sum(grouplist):
        print("评分函数中len(y_true)!=sum(group)")
        return
    start = 0
    sum_AUC = 0
    for group in grouplist:
        roc_auc = cal_AUC(y_true[start:start + group], y_pred[start:start + group])
        start = start + group
        sum_AUC = sum_AUC + roc_auc
    return sum_AUC


def QAUC_parallel(y_true, y_pred, group):
    groupnum = 4
    import math
    group_len = math.ceil(len(group) / groupnum)
    groups = [group[i * group_len:(i + 1) * group_len] for i in range(groupnum)]
    mycombines = []

    start = 0
    for agroup in groups:
        mycombinedata = []
        mycombinedata.append(agroup)
        mycombinedata.append(y_true[start:start + sum(agroup)])
        mycombinedata.append(y_pred[start:start + sum(agroup)])
        start = start + sum(agroup)
        mycombines.append(mycombinedata)

    sum_AUC_ = Parallel(n_jobs=groupnum)(delayed(sum_AUC)(mycombinedata) for mycombinedata in mycombines)

    return sum(sum_AUC_) / len(group)


class Evaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()
        self.interval = interval
        self.x_val, self.val_group, self.y_val = validation_data
        self.best_score = 0.
        self.best_epoch = 0
        self.best_auc = 0.0
        self.best_auc_epoch = 0
        self.auc_list = []
        self.q_auc_list = []

    def on_epoch_end(self, epoch, log={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.x_val, verbose=0, batch_size=batch_size)
            auc_score = roc_auc_score(self.y_val, y_pred)
            self.auc_list.append(auc_score)
            if auc_score > self.best_auc:
                self.best_auc = auc_score
                self.best_auc_epoch = epoch + 1

            score_parallel = QAUC_parallel(self.y_val, y_pred, self.val_group)
            self.q_auc_list.append(score_parallel)
            if score_parallel > self.best_score:
                self.best_score = score_parallel
                self.best_epoch = epoch + 1
            logger.info(f'Q_AUC = {score_parallel} epoch = {epoch + 1}')

            print('\n ROC_AUC - epoch:%d - score:%.6f' % (epoch + 1, auc_score))
            print('\n Q_AUC - epoch:%d - score:%.6f' % (epoch + 1, score_parallel))

    def get_best_score_epoch(self):
        return self.best_score, self.best_epoch

    def get_best_auc_score_epoch(self):
        return self.best_auc, self.best_auc_epoch

    def show_result_list(self):
        print('auc', self.auc_list)
        print('\n')
        print('qauc', self.q_auc_list)


def seq_padding(X, max_len):
    return [x + [PAD] * (max_len - len(x)) if len(x) < max_len else x[:max_len] for x in X]


class DataGenerator(keras.utils.Sequence):

    def __init__(self, word2id, text_data, lgb_data, batch_size=1024 * 5):
        self.batch_size = batch_size
        self.word2id = word2id

        self.text_data = text_data
        self.lgb_data = lgb_data

    def __len__(self):
        # 计算每一个epoch的迭代次数
        return math.ceil(len(self.text_data) / float(self.batch_size))

    def __getitem__(self, index):
        start = index * self.batch_size
        stop = (index + 1) * self.batch_size

        batch_lgb_df = self.lgb_data.iloc[start:stop]
        batch_text_df = self.text_data.iloc[start:stop]
        y = batch_text_df['label'].values

        train_lgb_input = [batch_lgb_df[feat].values for feat in used_lgb_dense_feature]

        Q = []
        D = []
        for query in batch_text_df['query']:
            query = query.split()
            Q.append([word2id[w] for w in query])

        for title in batch_text_df['title']:
            title = title.split()
            D.append([word2id[w] for w in title])

        Q_pad = seq_padding(Q, max_seq_len)
        D_pad = seq_padding(D, max_seq_len)
        return [np.array(Q_pad), np.array(D_pad)] + train_lgb_input, y


flag = 'train'
batch_size = 1024 * 5

PAD = 0
UNK = 1

W2V_DIM = 200

max_seq_len = 25
epochs = 20


def get_used_feature_names(featurecol_h5):
    features = []
    for k, v in featurecol_h5.items():
        features.extend(v)
    return features


featurecol_h5 = {

    'sim_feat': ['jaccard_q3_t3',
                 'jaccard_q3_t5',
                 'jaccard_q5_t5', 'levenshtein_q5_t5',
                 'jaccard_q5_t10', 'levenshtein_q5_t10',
                 'jaccard_q10_t10', 'levenshtein_q10_t10',
                 'jaccard_q15_t25', 'levenshtein_q15_t25',
                 'jaccard', 'levenshtein'],

    'len_feat': ["querykw_num", "titlekw_num"],

    "title_nunique_query": ["title_nunique_query"],
    "query_nunique_title": ["query_nunique_title"],

    'title_score_count_feat': ["title_score_count", "title_score_click_num"],
    'title_code_score_feat': ["title_code_score"],
    'title_convert_feat': ["title_code_convert", 'title_code_label_count'],

    'query_count': ["query_code_count"],
    'title_count': ["title_count"],

    "match_feat": ['count_match', 'blockcount_match', 'proximity', 'maxMatchBlockLen',
                   'q1_match_start', 'q1_match_end'],

    "BM25": ["BM25"],
}

othercols = ["titlekw_querykw_diff", "titlekw_querykw_rate"]


def reduce_mem_usage(D, verbose=True):
    start_mem = D.memory_usage().sum() / 1024 ** 2
    for c, d in zip(D.columns, D.dtypes):
        if d.kind == 'f':
            D[c] = pd.to_numeric(D[c], downcast='float')
        elif d.kind == 'i':
            D[c] = pd.to_numeric(D[c], downcast='signed')
    end_mem = D.memory_usage().sum() / 1024 ** 2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (
            start_mem - end_mem) / start_mem))
    return D


def ReadTrainData(datatype='train', nrows=100000000):
    DataSet = pd.read_hdf('/home/kesci/work/sunrui/post10kw/query_id_post10kw.h5', stop=nrows).reset_index(drop=True)
    label_set = pd.read_hdf('/home/kesci/work/label_train.h5', start=900000000, stop=900000000 + nrows).reset_index(
        drop=True)
    DataSet = pd.concat([DataSet, label_set], axis=1)

    featuremap_h5 = {
        'title_count': f'count_feature_{datatype}.h5',
        'query_count': f'query_count_all_{datatype}.h5',

        'cross_feat': f'cross_{datatype}_feat.h5',
        'editdistance_relativepos': f'editdistance_relativepos_{datatype}_feat.h5',
        'fuzz': f"fuzz_{datatype}_feat.h5",
        'len_feat': f'len_{datatype}_feat.h5',
        'NN_SIM': f'nn_sim_feature.h5',
        "title_nunique_query": f'nunique_feature_{datatype}.h5',
        "query_nunique_title": f'query_nunique_title_all_{datatype}.h5',
        'match_feat': f'query_match_{datatype}_feat.h5',
        'query_pos_feat': f'query_pos_{datatype}_feat.h5',
        'sen_dis': f"sen_dis_{datatype}_200.h5",
        'sen_dis2': f"sen_dis2_{datatype}_200.h5",
        'sim_feat': f'sim_{datatype}_feat.h5',
        'textpair': f"textpair_{datatype}_feat.h5",

        'tag_score_feat': f'tag_score_10foldtime_{datatype}_feat.h5',
        'title_score_count_feat': f'title_score_count_{datatype}_feat.h5',
        'title_code_score_feat': f'title_code_score_10foldtime_{datatype}_feat.h5',
        'title_convert_feat': f'title_convert_{datatype}.h5',
        'tag': f'tag_{datatype}.h5',
        "tag_convert_feat": f"tag_convert_{datatype}.h5",
        "query_convert": f"query_convert_{datatype}.h5",

        "BM25": f'BM25_{datatype}_feat.h5',
    }
    path_pre6kw = "/home/kesci/work/post_4kw_data/train_6kw/"
    path_post4kw = "/home/kesci/work/post_4kw_data/train/"
    for featurefile in featurecol_h5:
        print("读取", featuremap_h5[featurefile])
        feature_set1 = pd.read_hdf(path_pre6kw + featuremap_h5[featurefile], key='data',
                                   start=0, stop=60000000
                                   # start=0,stop=nrows
                                   )[featurecol_h5[featurefile]].reset_index(drop=True)
        # print(feature_set1.head(1))
        print("feature_set1 len:", feature_set1.__len__())
        feature_set2 = pd.read_hdf(path_post4kw + featuremap_h5[featurefile], key='data',
                                   start=0, stop=40000000
                                   # start=0,stop=nrows
                                   )[featurecol_h5[featurefile]].reset_index(drop=True)
        # print(feature_set2.head(1))
        print("feature_set2 len:", feature_set2.__len__())
        feature_set = pd.concat([feature_set1, feature_set2], axis=0).reset_index(drop=True)
        print("length:", feature_set.__len__())
        feature_set = reduce_mem_usage(feature_set, verbose=True)
        DataSet = pd.concat([DataSet, feature_set], axis=1)

    DataSet["titlekw_querykw_diff"] = DataSet["titlekw_num"] - DataSet["querykw_num"]
    DataSet["titlekw_querykw_rate"] = DataSet["titlekw_num"] / DataSet["querykw_num"]
    print("Data Read Finish!")
    return DataSet


if __name__ == "__main__":
    if flag == 'train':

        train_size = 98000000
        emb_mat = np.load('/home/kesci/work/sunrui/NN_second_2e/word2vec_fasttext_6kw_nn_sim.npy')

        text_ = pd.read_csv('/home/kesci/work/word2vec/post_10kw.csv').reset_index(drop=True)
        print(text_.shape)
        query_id = pd.read_hdf('/home/kesci/work/sunrui/post10kw/query_id_post10kw.h5').reset_index(drop=True)
        print(query_id.shape)

        label_ = pd.read_hdf('/home/kesci/work/label_train.h5', start=900000000, end=1000000000).reset_index(drop=True)
        print(label_.shape)

        text_data = pd.concat([text_, query_id, label_], axis=1)
        # 读取 lgb feature

        lgb_data = ReadTrainData(nrows=100000000)

        used_lgb_dense_feature = get_used_feature_names(featurecol_h5) + othercols

        print(used_lgb_dense_feature)
        lgb_data[used_lgb_dense_feature] = lgb_data[used_lgb_dense_feature].fillna(-1, )

        train_lgb_data = lgb_data[:train_size]
        val_lgb_data = lgb_data[train_size:]
        # 读取 lgb feature 完毕

        with open('/home/kesci/work/sunrui/NN_second_2e/second_6kw_nn_sim.pkl', 'rb') as f:
            word2id = pickle.load(f)

        val_text_data = text_data[train_size:]
        train_text_data = text_data[:train_size]

        Q_val = []
        D_val = []
        for query in val_text_data['query']:
            query = query.split()
            Q_val.append([word2id[w] for w in query])  # 没有命中就返回UNK

        for title in val_text_data['title']:
            title = title.split()
            D_val.append([word2id[w] for w in title])

        val_query_input = seq_padding(Q_val, max_seq_len)
        val_title_input = seq_padding(D_val, max_seq_len)

        Y_val = val_text_data['label'].values

        train_generator = DataGenerator(word2id=word2id, text_data=train_text_data,
                                        lgb_data=train_lgb_data, batch_size=batch_size)

        val_text_data['query_id_nums'] = val_text_data.groupby(['query_id'])['label'].transform('count')

        val_group_df = val_text_data[['query_id', 'query_id_nums']].drop_duplicates()
        val_group = val_group_df.query_id_nums.get_values()

        swa = SWA(checkpoint_dir='./sunrui/swa/', model_name='swa.model')

        clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                       step_size=2 * train_size // batch_size, mode='triangular')

        model = build_model(lstm_dim=64, emb_mat=emb_mat)
    
    
        filepath = "./sunrui/nn/gated_esim_{epoch:02d}_64_1e.hdf5"

        checkpoint = ModelCheckpoint(filepath, verbose=1)

        early_stopping = EarlyStopping(monitor='val_auc', patience=5, verbose=1, mode='max')

        val_lgb_input = [val_lgb_data[feat].values for feat in used_lgb_dense_feature]

        # train_model_input = [train_query_input, train_title_input] + train_lgb_input
        val_model_input = [val_query_input, val_title_input] + val_lgb_input

        eval_callback = Evaluation(
            validation_data=(
                val_model_input, val_group, Y_val))

        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[auc])
        model.summary()
        model.fit_generator(train_generator, epochs=epochs,
                            validation_data=(val_model_input, Y_val),
                            callbacks=[early_stopping, eval_callback, swa, clr, checkpoint], shuffle=True,
                            workers=2,
                            use_multiprocessing=True)
    else:
        pass
    


In [ ]:
# 以下是带有特征的esim网络 前1e数据的 隐层个数为64 我们选择了第1轮的结果 其实训练了4轮 
# 但是看第一轮的线下指标最好 

In [ ]:
import math
import os
import pickle
from collections import OrderedDict

import numpy as np
import pandas as pd
import tensorflow as tf
from keras import Input, Model
from keras.backend.tensorflow_backend import set_session
from keras.callbacks import EarlyStopping
from keras.layers import Embedding, Dense, Dropout, Lambda, concatenate, GlobalAveragePooling1D, subtract, multiply, \
    TimeDistributed, LSTM
from keras.regularizers import l2
from tensorflow import set_random_seed

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
set_session(sess)

seed = 2019

set_random_seed(seed)  # Tensorflow
np.random.seed(seed)  # NumPy

from keras.callbacks import Callback, ModelCheckpoint
import keras
import keras.backend as K
from keras.engine import Layer

import numpy as np

import logging

logger = logging.getLogger(__name__)
logger.setLevel(level=logging.INFO)
handler = logging.FileHandler("./sunrui/gated_esim_64_pre_1e_log.txt")  # 训练前1e的数据
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

console = logging.StreamHandler()
console.setLevel(logging.INFO)

logger.addHandler(handler)
logger.addHandler(console)


class DotProductAttention(Layer):
    """
    dot-product-attention mechanism, supporting masking
    """

    def __init__(self, return_attend_weight=False, keep_mask=True, **kwargs):
        self.return_attend_weight = return_attend_weight
        self.keep_mask = keep_mask
        self.supports_masking = True
        super(DotProductAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        input_shape_a, input_shape_b = input_shape

        if len(input_shape_a) != 3 or len(input_shape_b) != 3:
            raise ValueError('Inputs into DotProductAttention should be 3D tensors')

        if input_shape_a[-1] != input_shape_b[-1]:
            raise ValueError('Inputs into DotProductAttention should have the same dimensionality at the last axis')

    def call(self, inputs, mask=None):
        assert isinstance(inputs, list)
        inputs_a, inputs_b = inputs

        if mask is not None:
            mask_a, mask_b = mask
        else:
            mask_a, mask_b = None, None

        e = K.exp(K.batch_dot(inputs_a, inputs_b, axes=2))  # similarity between a & b

        # apply mask before normalization (softmax)
        if mask_a is not None:
            e *= K.expand_dims(K.cast(mask_a, K.floatx()), 2)
        if mask_b is not None:
            e *= K.expand_dims(K.cast(mask_b, K.floatx()), 1)

        e_b = e / K.cast(K.sum(e, axis=2, keepdims=True) + K.epsilon(), K.floatx())  # attention weight over b
        e_a = e / K.cast(K.sum(e, axis=1, keepdims=True) + K.epsilon(), K.floatx())  # attention weight over a

        if self.return_attend_weight:
            return [e_b, e_a]

        a_attend = K.batch_dot(e_b, inputs_b, axes=(2, 1))  # a attend to b
        b_attend = K.batch_dot(e_a, inputs_a, axes=(1, 1))  # b attend to a
        return [a_attend, b_attend]

    def compute_mask(self, inputs, mask=None):
        if self.keep_mask:
            return mask
        else:
            return [None, None]

    def compute_output_shape(self, input_shape):
        if self.return_attend_weight:
            input_shape_a, input_shape_b = input_shape
            return [(input_shape_a[0], input_shape_a[1], input_shape_b[1]),
                    (input_shape_a[0], input_shape_a[1], input_shape_b[1])]
        return input_shape


class SWA(Callback):
    def __init__(self, checkpoint_dir, model_name, swa_start=1):
        """
        :param checkpoint_dir: the directory where the model will be saved in
        :param model_name: the name of model we're training
        :param swa_start: the epoch when averaging begins. We generally pre-train the network for a certain amount of
                          epochs to start (swa_start > 1), as opposed to starting to track the average from the
                          very beginning.
        """
        super(SWA, self).__init__()
        self.checkpoint_dir = checkpoint_dir
        self.model_name = model_name
        self.swa_start = swa_start
        self.swa_model = None  # the model that we will use to store the average of the weights once SWA begins

    def on_train_begin(self, logs=None):
        self.epoch = 0
        self.swa_n = 0
        # self.swa_model = copy.deepcopy(self.model)  # make a copy of the model we're training
        # Note: I found deep copy of a model with customized layer would give errors
        self.swa_model = keras.models.clone_model(self.model)
        self.swa_model.set_weights(self.model.get_weights())  # see: https://github.com/keras-team/keras/issues/1765

    def on_epoch_end(self, epoch, logs=None):
        if (self.epoch + 1) >= self.swa_start:
            self.update_average_model()
            self.swa_n += 1

        self.epoch += 1

    def update_average_model(self):
        # update running average of parameters
        alpha = 1. / (self.swa_n + 1)
        for layer, swa_layer in zip(self.model.layers, self.swa_model.layers):
            weights = []
            for w1, w2 in zip(swa_layer.get_weights(), layer.get_weights()):
                weights.append((1 - alpha) * w1 + alpha * w2)
            swa_layer.set_weights(weights)

    def on_train_end(self, logs=None):
        print('Logging Info - Saving SWA model checkpoint: %s_swa.hdf5\n' % self.model_name)
        self.swa_model.save_weights(os.path.join(self.checkpoint_dir, '{}_swa.hdf5'.format(self.model_name)))
        print('Logging Info - SWA model Saved')


class CyclicLR(Callback):
    def __init__(
            self,
            base_lr=0.001,
            max_lr=0.006,
            step_size=2000.,
            mode='triangular',
            gamma=1.,
            scale_fn=None,
            scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        if mode not in ['triangular', 'triangular2',
                        'exp_range']:
            raise KeyError("mode must be one of 'triangular', "
                           "'triangular2', or 'exp_range'")
        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn is None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1 / (2. ** (x - 1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma ** x
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr is not None:
            self.base_lr = new_base_lr
        if new_max_lr is not None:
            self.max_lr = new_max_lr
        if new_step_size is not None:
            self.step_size = new_step_size
        self.clr_iterations = 0.

    def clr(self):
        cycle = np.floor(1 + self.clr_iterations / (2 * self.step_size))
        x = np.abs(self.clr_iterations / self.step_size - 2 * cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr - self.base_lr) * \
                   np.maximum(0, (1 - x)) * self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr - self.base_lr) * \
                   np.maximum(0, (1 - x)) * self.scale_fn(self.clr_iterations)

    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())

    def on_batch_end(self, epoch, logs=None):

        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1
        K.set_value(self.model.optimizer.lr, self.clr())

        self.history.setdefault(
            'lr', []).append(
            K.get_value(
                self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)


def get_dense_feature_inputs(dense_features_: list):
    dense_input = OrderedDict()
    for feature in dense_features_:
        dense_input[feature] = Input(shape=(1,), name=feature + '_input')
    return dense_input


def get_dense_feature_fc_list(dense_input_: OrderedDict, fc_dim=8, use_bias=True, l2_reg=1e-4):
    dense_input = list(dense_input_.values())
    fc_out_list = list(map(Dense(fc_dim, use_bias=use_bias, kernel_regularizer=l2(l2_reg)), dense_input))
    return fc_out_list


def build_model(lstm_dim=64, emb_mat=None):
    print('Build model...')

    query_input = Input(shape=(max_seq_len,))
    title_input = Input(shape=(max_seq_len,))

    embedding = Embedding(emb_mat.shape[0], W2V_DIM, weights=[emb_mat], trainable=False, mask_zero=True)

    query_emb = embedding(query_input)
    query_emb = Dropout(0.2)(query_emb)

    title_emb = embedding(title_input)
    title_emb = Dropout(0.2)(title_emb)

    bilstm_1 = LSTM(units=lstm_dim, return_sequences=True)

    query_hidden = bilstm_1(query_emb)
    title_hidden = bilstm_1(title_emb)

    query_attend, title_attend = DotProductAttention()([query_hidden, title_hidden])

    query_enhance = concatenate([query_hidden, query_attend, subtract([query_hidden, query_attend]),
                                 multiply([query_hidden, query_attend])])  # [?,25,256]

    title_enhance = concatenate([title_hidden, title_attend,
                                 subtract([title_hidden, title_attend]),
                                 multiply([title_hidden, title_attend])])  # [?,25,256]

    # inference composition
    feed_forward = TimeDistributed(Dense(units=lstm_dim, activation='relu'))

    bilstm_2 = LSTM(units=lstm_dim, return_sequences=True)

    query_compose = bilstm_2(feed_forward(query_enhance))  # [?,25,32]
    title_compose = bilstm_2(feed_forward(title_enhance))

    global_max_pooling = Lambda(lambda x: K.max(x, axis=1))  # GlobalMaxPooling1D didn't support masking
    query_avg = GlobalAveragePooling1D()(query_compose)
    query_max = global_max_pooling(query_compose)
    title_avg = GlobalAveragePooling1D()(title_compose)
    title_max = global_max_pooling(title_compose)

    lgb_dense_feature_input = get_dense_feature_inputs(used_lgb_dense_feature)

    dense_fc_list = get_dense_feature_fc_list(lgb_dense_feature_input)

    if len(dense_fc_list) > 1:
        dense_feature_concat = concatenate(dense_fc_list)
    else:
        dense_feature_concat = dense_fc_list[0]

    inference_compose = concatenate([query_avg, query_max, title_avg, title_max])

    # inference_compose = BatchNormalization()(inference_compose)  # 尝试

    dense_esim = Dense(units=lstm_dim)(inference_compose)

    dense_feature_gate = Dense(lstm_dim, activation='sigmoid')(dense_feature_concat)

    gated_esim = Lambda(lambda x: x[0] * x[1])([dense_esim, dense_feature_gate])

    gated_esim = Dense(lstm_dim, activation='elu')(gated_esim)
    model_dense_input = [lgb_dense_feature_input[feat] for feat in used_lgb_dense_feature]

    # gated_esim = BatchNormalization()(gated_esim)  # 加了BN 训练不稳定 val loss会跳 但是性能尚可
    # gated_esim = Dropout(0.1)(gated_esim)

    output = Dense(1, activation='sigmoid')(gated_esim)
    model = Model(inputs=[query_input, title_input] + model_dense_input, outputs=output)
    return model


def auc(y_true, y_pred):
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)


from joblib import Parallel, delayed
from sklearn.metrics import roc_auc_score


def cal_AUC(labels, prob):
    try:
        return roc_auc_score(labels, prob)
    except:
        return 0.5


# 计算各个组的qauc值
def sum_AUC(mycombinedata):
    grouplist = mycombinedata[0]
    y_true = mycombinedata[1]
    y_pred = mycombinedata[2]

    if len(y_true) != sum(grouplist):
        print("评分函数中len(y_true)!=sum(group)")
        return
    start = 0
    sum_AUC = 0
    for group in grouplist:
        roc_auc = cal_AUC(y_true[start:start + group], y_pred[start:start + group])
        start = start + group
        sum_AUC = sum_AUC + roc_auc
    return sum_AUC


def QAUC_parallel(y_true, y_pred, group):
    groupnum = 4
    import math
    group_len = math.ceil(len(group) / groupnum)
    groups = [group[i * group_len:(i + 1) * group_len] for i in range(groupnum)]
    mycombines = []

    start = 0
    for agroup in groups:
        mycombinedata = []
        mycombinedata.append(agroup)
        mycombinedata.append(y_true[start:start + sum(agroup)])
        mycombinedata.append(y_pred[start:start + sum(agroup)])
        start = start + sum(agroup)
        mycombines.append(mycombinedata)

    sum_AUC_ = Parallel(n_jobs=groupnum)(delayed(sum_AUC)(mycombinedata) for mycombinedata in mycombines)

    return sum(sum_AUC_) / len(group)


class Evaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()
        self.interval = interval
        self.x_val, self.val_group, self.y_val = validation_data
        self.best_score = 0.
        self.best_epoch = 0
        self.best_auc = 0.0
        self.best_auc_epoch = 0
        self.auc_list = []
        self.q_auc_list = []

    def on_epoch_end(self, epoch, log={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.x_val, verbose=0, batch_size=batch_size)
            auc_score = roc_auc_score(self.y_val, y_pred)
            self.auc_list.append(auc_score)
            if auc_score > self.best_auc:
                self.best_auc = auc_score
                self.best_auc_epoch = epoch + 1

            score_parallel = QAUC_parallel(self.y_val, y_pred, self.val_group)
            self.q_auc_list.append(score_parallel)
            if score_parallel > self.best_score:
                self.best_score = score_parallel
                self.best_epoch = epoch + 1
            logger.info(f'Q_AUC = {score_parallel} epoch = {epoch + 1}')

            print('\n ROC_AUC - epoch:%d - score:%.6f' % (epoch + 1, auc_score))
            print('\n Q_AUC - epoch:%d - score:%.6f' % (epoch + 1, score_parallel))

    def get_best_score_epoch(self):
        return self.best_score, self.best_epoch

    def get_best_auc_score_epoch(self):
        return self.best_auc, self.best_auc_epoch

    def show_result_list(self):
        print('auc', self.auc_list)
        print('\n')
        print('qauc', self.q_auc_list)


def seq_padding(X, max_len):
    return [x + [PAD] * (max_len - len(x)) if len(x) < max_len else x[:max_len] for x in X]


class DataGenerator(keras.utils.Sequence):

    def __init__(self, word2id, text_data, lgb_data, batch_size=1024 * 5):
        self.batch_size = batch_size
        self.word2id = word2id

        self.text_data = text_data
        self.lgb_data = lgb_data

    def __len__(self):
        # 计算每一个epoch的迭代次数
        return math.ceil(len(self.text_data) / float(self.batch_size))

    def __getitem__(self, index):
        start = index * self.batch_size
        stop = (index + 1) * self.batch_size

        batch_lgb_df = self.lgb_data.iloc[start:stop]
        batch_text_df = self.text_data.iloc[start:stop]
        y = batch_text_df['label'].values

        train_lgb_input = [batch_lgb_df[feat].values for feat in used_lgb_dense_feature]

        Q = []
        D = []
        for query in batch_text_df['query']:
            query = query.split()
            Q.append([word2id[w] for w in query])

        for title in batch_text_df['title']:
            title = title.split()
            D.append([word2id[w] for w in title])

        Q_pad = seq_padding(Q, max_seq_len)
        D_pad = seq_padding(D, max_seq_len)
        return [np.array(Q_pad), np.array(D_pad)] + train_lgb_input, y


flag = 'train'
batch_size = 1024 * 5

PAD = 0
UNK = 1

W2V_DIM = 200

max_seq_len = 25
epochs = 20


def get_used_feature_names(featurecol_h5):
    features = []
    for k, v in featurecol_h5.items():
        features.extend(v)
    return features


featurecol_h5 = {
    'sim_feat': ['jaccard_q3_t3',
                 'jaccard_q3_t5',
                 'jaccard_q5_t5', 'levenshtein_q5_t5',
                 'jaccard_q5_t10', 'levenshtein_q5_t10',
                 'jaccard_q10_t10', 'levenshtein_q10_t10',
                 'jaccard_q15_t25', 'levenshtein_q15_t25',
                 'jaccard', 'levenshtein'],

    'len_feat': ["querykw_num", "titlekw_num"],

    "title_nunique_query": ["title_nunique_query"],
    "query_nunique_title": ["query_nunique_title"],

    'title_score_count_feat': ["title_score_count", "title_score_click_num"],
    'title_code_score_feat': ["title_code_score"],
    'title_convert_feat': ["title_code_convert", 'title_code_label_count'],

    'query_count': ["query_code_count"],
    'title_count': ["title_count"],

    "match_feat": ['count_match', 'blockcount_match', 'proximity', 'maxMatchBlockLen',
                   'q1_match_start', 'q1_match_end'],

    "BM25": ["BM25"],
}

othercols = ["titlekw_querykw_diff", "titlekw_querykw_rate"]


def reduce_mem_usage(D, verbose=True):
    start_mem = D.memory_usage().sum() / 1024 ** 2
    for c, d in zip(D.columns, D.dtypes):
        if d.kind == 'f':
            D[c] = pd.to_numeric(D[c], downcast='float')
        elif d.kind == 'i':
            D[c] = pd.to_numeric(D[c], downcast='signed')
    end_mem = D.memory_usage().sum() / 1024 ** 2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (
            start_mem - end_mem) / start_mem))
    return D


def ReadData(datatype='train', nrows=1000000000):
    if datatype == 'train':
        id_feature = '/home/kesci/input/bytedance/train_final.csv'
        usecols = [0, 4]
        names = ['query_id', 'label']

        print("read ", id_feature)
        DataSet = pd.read_csv(id_feature,
                              header=None,
                              nrows=nrows,
                              usecols=usecols,
                              names=names
                              )
        path_h5 = "/home/kesci/work/pre_3billion_data/train/"
    elif datatype == 'test1':
        id_feature = '/home/kesci/input/bytedance/test_final_part1.csv'
        usecols = [0, 2]
        names = ['query_id', 'query_title_id']
        path_h5 = "/home/kesci/work/post_4kw_data/test1/"
        print("read ", id_feature)
        DataSet = pd.read_csv(id_feature,
                              header=None,
                              nrows=nrows,
                              usecols=usecols,
                              names=names
                              )
    elif datatype == 'test2':
        id_feature = '/home/kesci/input/bytedance/bytedance_contest.final_2.csv'
        usecols = [0, 2]
        names = ['query_id', 'query_title_id']
        path_h5 = "/home/kesci/work/post_4kw_data/test2/"
        print("read ", id_feature)
        DataSet = pd.read_csv(id_feature,
                              header=None,
                              nrows=nrows,
                              usecols=usecols,
                              names=names
                              )

    print("length:", DataSet.__len__())
    DataSet = reduce_mem_usage(DataSet, verbose=True)

    featuremap_h5 = {
        'cross_feat': path_h5 + f'cross_{datatype}_feat.h5',

        'query_pos_feat': path_h5 + f'query_pos_{datatype}_feat.h5',
        'title_pos_feat': path_h5 + f'title_pos_{datatype}_feat.h5',

        'match_feat': path_h5 + f'query_match_{datatype}_feat.h5',
        'editDistance_feat': path_h5 + f'editDistance_{datatype}_feat.h5',

        'sim_feat': path_h5 + f'sim_{datatype}_feat.h5',
        'tag_score_feat': path_h5 + f'tag_score_10foldtime_{datatype}_feat.h5',
        'title_score_count_feat': path_h5 + f'title_score_count_{datatype}_feat.h5',
        'title_code_score_feat': path_h5 + f'title_code_score_10foldtime_{datatype}_feat.h5',
        'title_convert_feat': path_h5 + f'title_convert_{datatype}.h5',
        'sif_feat': path_h5 + f'sif_{datatype}_post_4kw.h5',
        'len_feat': path_h5 + f'len_{datatype}_feat.h5',

        'title_count': path_h5 + f'count_feature_{datatype}.h5',
        'query_count': path_h5 + f'query_count_all_{datatype}.h5',

        "title_nunique_query": path_h5 + f'nunique_feature_{datatype}.h5',
        "query_nunique_title": path_h5 + f'query_nunique_title_all_{datatype}.h5',

        'tag': path_h5 + f'tag_{datatype}.h5',
        "tag_convert_feat": path_h5 + f"tag_convert_{datatype}.h5",
        "query_convert": path_h5 + f"query_convert_{datatype}.h5",

        "M_cosine": path_h5 + f"M_sim_{datatype}_feat.h5",
        "M_tfidf_cosine": path_h5 + f"M_tfidf_sim_{datatype}_feat.h5",
        "BM25": path_h5 + f'BM25_{datatype}_feat.h5',
        'NN_SIM': path_h5 + f'nn_sim_feature.h5',

        'editdistance_relativepos': path_h5 + f'editdistance_relativepos_{datatype}_feat.h5',
        'fuzz': path_h5 + f"fuzz_{datatype}_feat.h5",
        'textpair': path_h5 + f"textpair_{datatype}_feat.h5",

        'sen_dis': path_h5 + f"sen_dis_{datatype}_200.h5",
        'sen_dis2': path_h5 + f"sen_dis2_{datatype}_200.h5",
    }

    for featurefile in featurecol_h5:
        print("read ", featuremap_h5[featurefile])
        feature_set = pd.read_hdf(featuremap_h5[featurefile],
                                  key='data',
                                  start=0,
                                  stop=nrows)[featurecol_h5[featurefile]].reset_index(drop=True)
        print("length:", feature_set.__len__())
        # print(feature_set.head(1))
        # feature_set=reduce_mem_usage(feature_set, verbose=True)
        DataSet = pd.concat([DataSet, feature_set], axis=1)

    DataSet["titlekw_querykw_diff"] = DataSet["titlekw_num"] - DataSet["querykw_num"]
    DataSet["titlekw_querykw_rate"] = DataSet["titlekw_num"] / DataSet["querykw_num"]

    if "title_code_score" in DataSet.columns:
        DataSet.title_code_score = DataSet.title_code_score.fillna(0)
    if "tag_score" in DataSet.columns:
        DataSet.tag_score = DataSet.tag_score.fillna(0)

    DataSet = reduce_mem_usage(DataSet, verbose=True)
    print("Data Read Finish!")
    return DataSet


if __name__ == "__main__":
    if flag == 'train':

        train_size = 98000000
        emb_mat = np.load('/home/kesci/work/sunrui/NN_second_2e/word2vec_fasttext_6kw_nn_sim.npy')
        
        lgb_data = ReadData(datatype='train', nrows=100000000)
        used_lgb_dense_feature = get_used_feature_names(featurecol_h5) + othercols

        print(used_lgb_dense_feature)

        text_data = pd.read_csv('/home/kesci/input/bytedance/train_final.csv', usecols=[0, 1, 3, 4], header=None,
            names=['query_id', 'query', 'title', 'label'], nrows=100000000)
        # 读取 lgb feature
        print(text_data.shape)
        lgb_data[used_lgb_dense_feature] = lgb_data[used_lgb_dense_feature].fillna(-1, )

        train_lgb_data = lgb_data[:train_size]
        val_lgb_data = lgb_data[train_size:]
        # 读取 lgb feature 完毕

        with open('/home/kesci/work/sunrui/NN_second_2e/second_6kw_nn_sim.pkl', 'rb') as f:
            word2id = pickle.load(f)

        val_text_data = text_data[train_size:]
        train_text_data = text_data[:train_size]

        Q_val = []
        D_val = []
        for query in val_text_data['query']:
            query = query.split()
            Q_val.append([word2id[w] for w in query])  # 没有命中就返回UNK

        for title in val_text_data['title']:
            title = title.split()
            D_val.append([word2id[w] for w in title])

        val_query_input = seq_padding(Q_val, max_seq_len)
        val_title_input = seq_padding(D_val, max_seq_len)

        Y_val = val_text_data['label'].values

        train_generator = DataGenerator(word2id=word2id, text_data=train_text_data,
                                        lgb_data=train_lgb_data, batch_size=batch_size)

        val_text_data['query_id_nums'] = val_text_data.groupby(['query_id'])['label'].transform('count')

        val_group_df = val_text_data[['query_id', 'query_id_nums']].drop_duplicates()
        val_group = val_group_df.query_id_nums.get_values()

        swa = SWA(checkpoint_dir='./sunrui/swa/', model_name='swa.model')

        clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                       step_size=2 * train_size // batch_size, mode='triangular')

        model = build_model(lstm_dim=64, emb_mat=emb_mat)

        filepath = "/home/kesci/work/sunrui/nn/gated_pre_1e/gated-{epoch:02d}_esim_64_pre1e.hdf5"
        checkpoint = ModelCheckpoint(filepath, verbose=1)

        early_stopping = EarlyStopping(monitor='val_auc', patience=5, verbose=1, mode='max')

        val_lgb_input = [val_lgb_data[feat].values for feat in used_lgb_dense_feature]

        # train_model_input = [train_query_input, train_title_input] + train_lgb_input
        val_model_input = [val_query_input, val_title_input] + val_lgb_input

        eval_callback = Evaluation(
            validation_data=(
                val_model_input, val_group, Y_val))

        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[auc])
        model.summary()
        model.fit_generator(train_generator, epochs=epochs,
                            validation_data=(val_model_input, Y_val),
                            callbacks=[early_stopping, eval_callback, swa, clr, checkpoint], shuffle=True,
                            workers=2,
                            use_multiprocessing=True)
    else:
        pass